# Parsing author data from csv

You will need the following in your LaTeX header for the hyperlinks and ORCiD symbols to work:
```
\usepackage{graphicx}	% Including figure files
\usepackage{amsmath}	% Advanced maths commands
\usepackage{scalerel}
\usepackage{tikz}
\usetikzlibrary{svg.path}

\definecolor{orcidlogocol}{HTML}{A6CE39}
\tikzset{
  orcidlogo/.pic={
    \fill[orcidlogocol] svg{M256,128c0,70.7-57.3,128-128,128C57.3,256,0,198.7,0,128C0,57.3,57.3,0,128,0C198.7,0,256,57.3,256,128z};
    \fill[white] svg{M86.3,186.2H70.9V79.1h15.4v48.4V186.2z}
                 svg{M108.9,79.1h41.6c39.6,0,57,28.3,57,53.6c0,27.5-21.5,53.6-56.8,53.6h-41.8V79.1z M124.3,172.4h24.5c34.9,0,42.9-26.5,42.9-39.7c0-21.5-13.7-39.7-43.7-39.7h-23.7V172.4z}
                 svg{M88.7,56.8c0,5.5-4.5,10.1-10.1,10.1c-5.6,0-10.1-4.6-10.1-10.1c0-5.6,4.5-10.1,10.1-10.1C84.2,46.7,88.7,51.3,88.7,56.8z};
  }
}

\newcommand\orcidicon[1]{\href{https://orcid.org/#1}{\mbox{\scalerel*{
\begin{tikzpicture}[xscale=0.1,yscale=-0.1,transform shape]
\pic{orcidlogo};
\end{tikzpicture}
}{|}}}}

\usepackage{hyperref}   %Needs to be at the end for some reason
```


In [2]:
import pandas as pd
import numpy as np

In [2]:
second_author = True
high_ranked_authors = np.arange(2,13)

In [22]:
df=pd.read_csv("TOI 755 author list.tsv",delimiter='\t',comment='#')

In [4]:
all_affils=[]
df['N_AFFILS']=np.tile(-99,len(df))

start_latex=""

In [5]:
all_affils+=[df.loc[0,'AFFILIATION 1 (Latex-compatible)']]
naffil=str(1)
df.loc[0,'N_AFFILS']="\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
print(df.loc[0,'N_AFFILS'])
if not pd.isnull(df.loc[0,'AFFILIATION 2 (Latex-compatible)']):
    all_affils+=[df.loc[0,'AFFILIATION 2 (Latex-compatible)']]
    naffil=str(2)
    df.loc[0,'N_AFFILS']+=","+"\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
    print(df.loc[0,'N_AFFILS'])

\hyperlink{affil_1}{1}
\hyperlink{affil_1}{1},\hyperlink{affil_2}{2}


In [6]:
#First author info:
start_latex+='\\author['+df.loc[0,'NAME']+" et al.]{"
if not pd.isnull(df.loc[0,'ORCiD']):
    orc=",\\orcidicon{"+str(df.loc[0,'ORCiD'])+"}"
else:
    orc=""
start_latex+='\\parbox{\\textwidth}{'+df.loc[0,'NAME']+'\\textsuperscript{'+df.loc[0,'N_AFFILS']+orc+'}\\thanks{E-mail: '+df.loc[0,"EMAIL (optional)"]+'}, \n'

In [7]:
if second_author:
    #Creating affiliations:
    if df.loc[1,'AFFILIATION 1 (Latex-compatible)'] in all_affils:
        naffil=str(all_affils.index(df.loc[1,'AFFILIATION 1 (Latex-compatible)'])+1)
        df.loc[1,'N_AFFILS']="\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
    else:
        naffil=str(len(all_affils)+1)
        df.loc[1,'N_AFFILS']="\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
        all_affils+=[df.loc[1,'AFFILIATION 1 (Latex-compatible)']]
    if not pd.isnull(df.loc[1,'AFFILIATION 2 (Latex-compatible)']) and df.loc[1,'AFFILIATION 2 (Latex-compatible)'] in all_affils:
        naffil=str(all_affils.index(df.loc[1,'AFFILIATION 2 (Latex-compatible)'])+1)
        df.loc[1,'N_AFFILS']+=","+"\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
    elif not pd.isnull(df.loc[1,'AFFILIATION 2 (Latex-compatible)']):
        naffil=str(len(all_affils)+1)
        df.loc[1,'N_AFFILS']+=","+"\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
        all_affils+=[df.loc[1,'AFFILIATION 2 (Latex-compatible)']]
    
    #Making first name into initial:
    if '.' not in df.loc[1,'NAME'].split(' ')[0]:
        name=df.loc[1,'NAME'].split(' ')[0][0]+'. '+' '.join(df.loc[1,'NAME'].split(' ')[1:])
    else:
        name=df.loc[1,'NAME']
    if len(name.split('. '))>2:
        name='.'.join(name.split('. ')[:-1])+'. '+name.split('. ')[-1]
    name=name.replace(' ','~')
    
    if not pd.isnull(df.loc[1,'ORCiD']):
        orc=",\\orcidicon{"+str(df.loc[1,'ORCiD'])+"}"
    else:
        orc=""
    #second author info:
    start_latex+=name+'\\textsuperscript{'+df.loc[1,'N_AFFILS']+orc+'}, %'
    if not pd.isnull(df.loc[1,"EMAIL (optional)"]):
        start_latex+=' '+df.loc[1,"EMAIL (optional)"]
    if not pd.isnull(df.loc[1,"CONTRIBUTION"]):
        start_latex+=' '+df.loc[1,"CONTRIBUTION"]
    start_latex+='\n'    

In [8]:
#Initial alphabetical list:
high_ranks=df.loc[high_ranked_authors]
high_ranks=high_ranks.iloc[np.argsort([h.split(' ')[-1][0] for h in high_ranks['NAME']])]

for n,auth in high_ranks.iterrows():
    #Creating affiliations:
    if auth['AFFILIATION 1 (Latex-compatible)'] in all_affils:
        naffil=str(all_affils.index(auth['AFFILIATION 1 (Latex-compatible)'])+1)
        df.loc[n,'N_AFFILS']="\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
    else:
        naffil=str(len(all_affils)+1)
        df.loc[n,'N_AFFILS']="\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
        all_affils+=[auth['AFFILIATION 1 (Latex-compatible)']]
    if not pd.isnull(auth['AFFILIATION 2 (Latex-compatible)']) and auth['AFFILIATION 2 (Latex-compatible)'] in all_affils:
        naffil=str(all_affils.index(auth['AFFILIATION 2 (Latex-compatible)'])+1)
        df.loc[n,'N_AFFILS']+=","+"\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
    elif not pd.isnull(auth['AFFILIATION 2 (Latex-compatible)']):
        naffil=str(len(all_affils)+1)
        df.loc[n,'N_AFFILS']+=","+"\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
        all_affils+=[auth['AFFILIATION 2 (Latex-compatible)']]
    
    #Making first name into initial:
    if '.' not in auth['NAME'].split(' ')[0]:
        name=auth['NAME'].split(' ')[0][0]+'. '+' '.join(auth['NAME'].split(' ')[1:])
    else:
        name=auth['NAME']
    if len(name.split('. '))>2:
        name='.'.join(name.split('. ')[:-1])+'. '+name.split('. ')[-1]
    name=name.replace(' ','~')
    
    #adding ORCiD:
    if not pd.isnull(auth['ORCiD']):
        orc=",\\orcidicon{"+str(auth['ORCiD'])+"}"
    else:
        orc=""
    
    #adding author info to latex:
    start_latex+=name+'\\textsuperscript{'+df.loc[n,'N_AFFILS']+orc+'}, %'
    if not pd.isnull(auth["EMAIL (optional)"]):
        start_latex+=' '+auth["EMAIL (optional)"]
    if not pd.isnull(auth["CONTRIBUTION"]):
        start_latex+=' '+auth["CONTRIBUTION"]
    start_latex+='\n'

In [9]:
# The rest:
low_ranks=df.loc[(1+high_ranked_authors[-1]):]
low_ranks=low_ranks.iloc[np.argsort([h.split(' ')[-1][0] for h in low_ranks['NAME']])]

for n,auth in low_ranks.iterrows():
    #Creating affiliations:
    if auth['AFFILIATION 1 (Latex-compatible)'] in all_affils:
        naffil=str(all_affils.index(auth['AFFILIATION 1 (Latex-compatible)'])+1)
        df.loc[n,'N_AFFILS']="\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
    else:
        naffil=str(len(all_affils)+1)
        df.loc[n,'N_AFFILS']="\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
        all_affils+=[auth['AFFILIATION 1 (Latex-compatible)']]
    if not pd.isnull(auth['AFFILIATION 2 (Latex-compatible)']) and auth['AFFILIATION 2 (Latex-compatible)'] in all_affils:
        naffil=str(all_affils.index(auth['AFFILIATION 2 (Latex-compatible)'])+1)
        df.loc[n,'N_AFFILS']+=","+"\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
    elif not pd.isnull(auth['AFFILIATION 2 (Latex-compatible)']):
        naffil=str(len(all_affils)+1)
        df.loc[n,'N_AFFILS']+=","+"\\hyperlink{affil_"+naffil+"}{"+naffil+"}"
        all_affils+=[auth['AFFILIATION 2 (Latex-compatible)']]
    
    #Making first name into initial:
    if '.' not in auth['NAME'].split(' ')[0]:
        name=auth['NAME'].split(' ')[0][0]+'. '+' '.join(auth['NAME'].split(' ')[1:])
    else:
        name=auth['NAME']
    if len(name.split('. '))>2:
        name='.'.join(name.split('. ')[:-1])+'. '+name.split('. ')[-1]
    name=name.replace(' ','~')
        
    #adding ORCiD:
    if not pd.isnull(auth['ORCiD']):
        orc=",\\orcidicon{"+str(auth['ORCiD'])+"}"
    else:
        orc=""
    
    #adding author info to latex:
    start_latex+=name+'\\textsuperscript{'+df.loc[n,'N_AFFILS']+orc+'}, %'
    if not pd.isnull(auth["EMAIL (optional)"]):
        start_latex+=' '+auth["EMAIL (optional)"]
    if not pd.isnull(auth["CONTRIBUTION"]):
        start_latex+=' '+auth["CONTRIBUTION"]
    start_latex+='\n'


In [10]:
start_latex+="}\\\\\n\\vspace{0.4cm}\n\\parbox{\\textwidth}{\nThe authors' affiliations are shown in Appendix \\ref{sec:affiliations}.}\n}"

In [11]:
print(start_latex)

\author[H.P. Osborn et al.]{\parbox{\textwidth}{H.P. Osborn\textsuperscript{\hyperlink{affil_1}{1},\hyperlink{affil_2}{2},\orcidicon{0000-0002-4047-4724}}\thanks{E-mail: hugh.osborn@space.unibe.ch}, 
D.J.~Armstrong\textsuperscript{\hyperlink{affil_3}{3},\hyperlink{affil_4}{4},\orcidicon{0000-0002-5080-4117}}, % d.j.armstrong@warwick.ac.uk HARPS NCORES PI
V.~Adibekyan\textsuperscript{\hyperlink{affil_5}{5},\orcidicon{0000-0002-0601-6199}}, % Vardan.Adibekyan@astro.up.pt Stellar abundances and ages
K.A.~Collins\textsuperscript{\hyperlink{affil_6}{6},\orcidicon{0000-0001-6588-9574}}, % karen.collins@cfa.harvard.edu SG1 coordination and data analysis
E.~Delgado-Mena\textsuperscript{\hyperlink{affil_5}{5},\orcidicon{0000-0003-4434-2195}}, % elisa.delgado@astro.up.pt Stellar abundances and ages
S.B.~Howell\textsuperscript{\hyperlink{affil_7}{7},\orcidicon{0000-0002-2532-2853}}, % steve.b.howell@nasa.gov Speckle observing, data reduction, analysis
C.~Hellier\textsuperscript{\hyperlink{affil_8

In [12]:
aff_box=""

In [13]:
for n,row in enumerate(all_affils):
    aff_box+="\\textsuperscript{\\hypertarget{affil_"+str(n+1)+"}{"+str(n+1)+"}} "
    aff_box+=row
    if row[-2]!='\\':
        aff_box+="\\\\"
    aff_box+="\n"

In [14]:
print(aff_box)

\textsuperscript{\hypertarget{affil_1}{1}} NCCR/PlanetS, Centre for Space \& Habitability, University of Bern, Bern, Switzerland\\
\textsuperscript{\hypertarget{affil_2}{2}} Department of Physics and Kavli Institute for Astrophysics and Space Research, Massachusetts Institute of Technology, Cambridge, MA 02139, USA\\
\textsuperscript{\hypertarget{affil_3}{3}} Centre for Exoplanets and Habitability, University of Warwick, Gibbet Hill Road, Coventry, CV4 7AL, UK\\
\textsuperscript{\hypertarget{affil_4}{4}} Department of Physics, University of Warwick, Gibbet Hill Road, Coventry CV4 7AL, UK\\
\textsuperscript{\hypertarget{affil_5}{5}} Instituto de Astrof\'isica e Ci\^encias do Espa\c{c}o, Universidade do Porto, CAUP, Rua das Estrelas, 4150-762 Porto, Portugal\\
\textsuperscript{\hypertarget{affil_6}{6}} Harvard-Smithsonian Center for Astrophysics, 60 Garden St, Cambridge, MA, 02138, USA\\
\textsuperscript{\hypertarget{affil_7}{7}} NASA Ames Research Center,Moffett Field, CA 94035, USA\\
\

In [15]:
acks=[]
for n,row in df.iterrows():
    if type(row['ACKNOWLEDGEMENT (Latex-compatible)'])==str:
        ack=row['ACKNOWLEDGEMENT (Latex-compatible)']
        ack=ack.split('\\')[0] if '\\' in ack else ack 
        acks+=[ack+"\\\\ %"+row['NAME']]
print('\n'.join(acks))

DJA acknowledges support from the STFC via an Ernest Rutherford Fellowship (ST/R00384X/1).\\ %D.J. Armstrong
This study is based on observations collected at the European Southern Observatory under ESO programme 1102.C-0249.\\ %L.D. Nielsen
This work makes use of observations from the LCOGT network.\\ %Karen A. Collins
V.A., E.D.M., N.C.S. acknowledge support by FCT - Funda\\ %V. Adibekyan
V.A. and E.D.M. further acknowledge the support from FCT through Investigador FCT contracts IF/00650/2015/CP1273/CT0001 and IF/00849/2015/CP1273/CT0003.\\ %E. Delgado-Mena
(Some of the) Observations in the paper made use of the High-Resolution Imaging instrument(s) ‘Alopeke (and/or Zorro). ‘Alopeke (and/or Zorro) was funded by the NASA Exoplanet Exploration Program and built at the NASA Ames Research Center by Steve B. Howell, Nic Scott, Elliott P. Horch, and Emmett Quigley. Data were reduced using a software pipeline originally written by Elliott Horch and Mark Everett. ‘Alopeke (and/or Zorro) was m